### **1.Preprocessing**

In [42]:
import re

# Load the file content
file_path = 'nfr.txt'
with open(file_path, 'r') as file:
    content = file.read()

# Remove non-ASCII characters
cleaned_content = re.sub(r'[^\x00-\x7F]', '', content)

# Remove extra spaces between words on each line
cleaned_content = re.sub(r'[ \t]+', ' ', cleaned_content)
# Ensure there is only one new line, and it maintains spaces after the new line
cleaned_content = re.sub(r'\n+', '\n', cleaned_content)

# save the cleaned content to a new file
with open('cleaned_nfr.txt', 'w') as file:
    file.write(cleaned_content)


### **2.categorization**

In [43]:
from collections import defaultdict
requirements = defaultdict(list)


for line in cleaned_content.splitlines():
    if line.strip():
        parts = line.split(":", 1)
        if len(parts) == 2:
            prefix = parts[0].strip()
            content = parts[1].strip()

            # Add the content to the appropriate category in the dictionary
            requirements[prefix].append(content)

# number of requirements in each category
for key, reqs in requirements.items():
    print(f"{key}: {len(reqs)} requirements")

PE: 54 requirements
LF: 38 requirements
US: 67 requirements
A: 21 requirements
SE: 66 requirements
F: 255 requirements
L: 13 requirements
O: 62 requirements
SC: 21 requirements
FT: 10 requirements
MN: 17 requirements
PO: 1 requirements


In [44]:
category_key = "PE"  # Replace with required key
if category_key in requirements:
    category_requirements = requirements[category_key]

    print(f"Requirements for category '{category_key}':")
    for req in category_requirements:
        print(f"- {req}")

Requirements for category 'PE':
- The system shall refresh the display every 60 seconds.
- The product shall respond fast to keep up-to-date data in the display.
- The search for recycled parts shall take no longer than 15 seconds. The search results shall be returned in under 15 seconds.
- The search for the preferred repair facility shall take no longer than 8 seconds. The preferred repair facility is returned within 8 seconds
- The recycled parts audit report shall be returned to the user within 10 seconds. The audit report shall be returned within 10 seconds.
- The preferred repair facility ratings shall be saved within 5 seconds. The save shall occur within 5 seconds.
- The response shall be fast enough to avoid interrupting the users flow of thought. 90% of tasks shall complete within 5 seconds. 98% of tasks shall complete within 8 seconds.
- Recreation of Inventory Quantity Adjustment for the period of previous 30 days shall take no more than 30min.
- The System shall allow for 

### **3.Text Extraction**

In [45]:
import re
import json

# Rules for extracting entities
def extract_actor(text):
    match = re.search(r'^(.*?)\b(shall|should|will|must|is)\b', text, re.IGNORECASE)
    return match.group(1).strip() if match else None

def extract_action(text):
    match = re.search(r'\b(shall|should|will|must|is)\b(.*?)\b(every|to|if|when|only|for|than|in|within|during|\.)', text, re.IGNORECASE)
    return match.group(2).strip() if match else None

def extract_condition(text):
    match = re.search(r'\b(every|to|if|when|only|for|than|in|within|during)\b(.*?)(\.)', text, re.IGNORECASE)
    return match.group(2).strip() if match else None

def extract_outcome(text):
    lines = text.split('\n')

    if len(lines) > 1:
        return lines[1].strip()
    else:
        outcome = re.sub(r'\b(shall|will|must|is|is expected)\b', 'should', lines[0], flags=re.IGNORECASE)
        return outcome.strip()

def extract_entities(requirement):
    return {
        "requirement": requirement,
        "actor": extract_actor(requirement),
        "action": extract_action(requirement),
        "condition": extract_condition(requirement),
        "outcome": extract_outcome(requirement)
    }

# Process all categories from the requirements dictionary
def process_requirements(requirements_dict):
    extracted_data = {}
    for category, requirements in requirements_dict.items():
        extracted_data[category] = [extract_entities(req) for req in requirements]
    return extracted_data



# Convert default dictionary to a regular dictionary for processing
requirements_dict = dict(requirements)

# Extract entities for all categories
extracted_data = process_requirements(requirements_dict)

# Save the extracted data to a JSON file
with open('extracted_entities.json', 'w') as json_file:
    json.dump(extracted_data, json_file, indent=4)

print("Entities have been extracted and saved to 'extracted_entities.json'.")


Entities have been extracted and saved to 'extracted_entities.json'.


###**4.Testcase generation**

In [46]:
import json

# Load the JSON file content
file_path = '/content/extracted_entities.json'
with open(file_path, 'r') as f:
    extracted_entities = json.load(f)

# Templates for interactive test steps
action_templates = {
    "refresh": {
        "step": "Click the 'Refresh' button located on the dashboard.",
        "condition": "Check whether the display is updated within {condition}."
    },
    "search": {
        "step": "Enter the search criteria in the search bar and click the 'Search' button.",
        "condition": "Ensure the search results are returned within {condition}."
    },
    "respond": {
        "step": "Perform the action that requires a response.",
        "condition": "Verify that the system responds within {condition}."
    },
    "save": {
        "step": "Enter the necessary data and click the 'Save' button.",
        "condition": "Ensure that the data is saved successfully within {condition}."
    },
    "submit": {
        "step": "Fill out the required fields and click the 'Submit' button.",
        "condition": "Verify that the form submission is processed within {condition}."
    },
    "allow": {
        "step": "Verify that the system allows multiple users to perform actions concurrently.",
        "condition": "Check that the system supports {condition} without any issues."
    },
    "activate": {
        "step": "Navigate to the Administration section and activate the pre-paid card.",
        "condition": "Ensure the activation is completed within {condition}."
    },
    "check": {
        "step": "Enter the PIN number and check the status of the prepaid card.",
        "condition": "Ensure that the status is retrieved within {condition}."
    },
    "register": {
        "step": "Fill in the registration form and submit the registration request.",
        "condition": "Verify that the registration is completed within {condition}."
    },
    "log": {
        "step": "Enter email and password, and click 'Log in'.",
        "condition": "Ensure that the login is successful within {condition}."
    },
    "cancel": {
        "step": "Initiate the account cancellation process.",
        "condition": "Verify that the account is marked as inactive and the user is logged out within {condition}."
    },
    "de-activate": {
        "step": "Navigate to the admin portal and deactivate the customer account.",
        "condition": "Check whether the customer is blocked from accessing the system within {condition}."
    },
    "add/remove": {
        "step": "Use the admin tools to add or remove items such as movies or categories.",
        "condition": "Ensure the changes are applied to the website within {condition}."
    },
    "access": {
        "step": "Log in to the admin panel and access sales and usage statistics.",
        "condition": "Verify that the stats are loaded within {condition}."
    },
    "generate": {
        "step": "Initiate the process to generate the necessary files (e.g., PINs or reports).",
        "condition": "Ensure the file generation completes within {condition}."
    },
    "view": {
        "step": "Navigate to the interface that displays the information.",
        "condition": "Check whether the data is displayed properly within {condition}."
    },
    "buffer": {
        "step": "Start the video stream.",
        "condition": "Ensure buffering time is no longer than {condition}."
    },
    "protect": {
        "step": "Verify that no information that reveals the identity of a player is accessible.",
        "condition": "Ensure data protection complies with the requirement."
    }
}

def generate_test_case_title(actor, action, condition):
    # Format title using actor, action, and condition
    if actor and action and condition:
        return f"Verify that {actor.lower()} can {action} within {condition}"
    return "Incomplete test case details"

def generate_preconditions(action):
    preconditions = ["1. Navigate to the website/application.",
                      "2. Login with valid credentials"]

    return preconditions

def generate_test_steps(action, condition):
    if not action:
        return ["No action specified."]
    if not condition:
        return ["No condition specified."]

    # Convert action to lowercase for case-insensitive comparison
    action_lower = action.lower()

    # Match the action to a template and generate relevant steps
    for key_action in action_templates.keys():
        # Check if the key_action phrase is in the action string
        if key_action in action_lower:
            template = action_templates[key_action]
            steps = [f"1. {template['step']}", f"2. {template['condition'].format(condition=condition)}"]
            return steps

    # Default action if no template matches
    return [f"1. Perform the action: {action}", f"2. Check whether {condition} occurs as expected."]

def generate_expected_outcome(outcome):
    if not outcome:
        return ["No expected outcome specified."]
    return outcome.split(". ")

def generate_test_case(requirement):
    actor = requirement.get("actor")
    action = requirement.get("action")
    condition = requirement.get("condition")
    outcome = requirement.get("outcome")

    return {
        "Title": generate_test_case_title(actor, action, condition),
        "Preconditions": generate_preconditions(action),
        "Test Steps": generate_test_steps(action, condition),
        "Expected Outcome": generate_expected_outcome(outcome)
    }

def process_requirements(extracted_entities):
    test_cases = []
    for category, requirements in extracted_entities.items():
        for requirement in requirements:
            test_case = generate_test_case(requirement)
            test_cases.append(test_case)
    return test_cases

# Process the requirements and generate test cases
test_cases = process_requirements(extracted_entities)

# Output the total number of test cases
print(f"Total test cases generated: {len(test_cases)}")

# Output the first 100 test cases in a clear format
for i, case in enumerate(test_cases[:100]):
    print(f"Test Case {i + 1}:")
    print(f"Title: {case['Title']}")
    print("Preconditions:")
    for precondition in case['Preconditions']:
        print(f"- {precondition}")
    print("Test Steps:")
    for step in case['Test Steps']:
        print(f"- {step}")
    print("Expected Outcome:")
    for outcome in case['Expected Outcome']:
        print(f"- {outcome}")
    print("-" * 100)


Total test cases generated: 625
Test Case 1:
Title: Verify that the system can refresh the display within 60 seconds
Preconditions:
- 1. Navigate to the website/application.
- 2. Login with valid credentials
Test Steps:
- 1. Click the 'Refresh' button located on the dashboard.
- 2. Check whether the display is updated within 60 seconds.
Expected Outcome:
- The system should refresh the display every 60 seconds.
----------------------------------------------------------------------------------------------------
Test Case 2:
Title: Verify that the product can respond fast within keep up-to-date data in the display
Preconditions:
- 1. Navigate to the website/application.
- 2. Login with valid credentials
Test Steps:
- 1. Perform the action that requires a response.
- 2. Verify that the system responds within keep up-to-date data in the display.
Expected Outcome:
- The product should respond fast to keep up-to-date data in the display.
------------------------------------------------------

### **5.Evaluation**

In [47]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assume binary labels for match (1) and mismatch (0) for each attribute
y_true = [1, 1, 0, 1]  # True labels
y_pred = [1, 0, 0, 1]  # Predicted labels

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Precision: 1.00
Recall: 0.67
F1 Score: 0.80


In [48]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.75
